In [1]:
import sys
sys.path.append("../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from PC import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
def evaluatePC(model, loader, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                         neural_lr_decay_multiplier,
                         neural_dynamic_iterations, device, printing = True):
    # Evaluate the model on a dataloader with T steps for the dynamics
    #model.eval()
    correct=0
    phase = 'Train' if loader.dataset.train else 'Test'
    
    for x, y in loader:
        x = activation_inverse(2*x.view(x.size(0),-1).to(device).T - 1, "sigmoid")
#         x = activation_inverse(x.view(x.size(0),-1).T, "sigmoid").to(device)
#         x = x.view(x.size(0),-1).T.to(device)
        y = y.to(device)
        
        neurons = model.fast_forward(x)
        
#         # dynamics for T time steps
#         neurons = model.run_neural_dynamics(x, y_one_hot, neurons, neural_lr_start, neural_lr_stop, 
#                                             neural_lr_rule,
#                                             neural_lr_decay_multiplier, neural_dynamic_iterations, 0, "test")
        pred = torch.argmax(neurons[-1], dim=0).squeeze()  # in this case prediction is done directly on the last (output) layer of neurons
        correct += (y == pred).sum().item()

    acc = correct/len(loader.dataset) 
    if printing:
        print(phase+' accuracy :\t', acc)   
    return acc

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.0,), std=(1.0,))])

mnist_dset_train = torchvision.datasets.MNIST('./data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('./data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [5]:
activation_type = "sigmoid"
architecture = [784, 500, 500, 10]

x,y = next(iter(train_loader))
x = x.view(x.size(0),-1).to(device).T
y_one_hot = F.one_hot(y, 10).to(device).T

neural_lr_start = 0.1
neural_lr_stop = 0.05
neural_lr_rule = "divide_by_slow_loop_index"
neural_lr_decay_multiplier = 0.01
neural_dynamic_iterations = 50

lr_start = {'ff' : 0.0025}

model = SupervisedPredictiveCoding(architecture, activation_type)

In [6]:
x, y = next(iter(train_loader))
x, y = x.to(device), y.to(device)
x = activation_inverse(x.view(x.size(0),-1).T, "sigmoid")
x.shape

torch.Size([784, 20])

In [7]:
trn_acc_list = []
tst_acc_list = []

n_epochs = 50
lr = lr_start
for epoch_ in range(n_epochs):
    lr = {'ff' : lr_start['ff'] * (0.9)**epoch_}
    for idx, (x, y) in tqdm(enumerate(train_loader)):
#         x = x.view(x.size(0),-1).T#.to(device)
        x, y = x.to(device), y.to(device)
        x = activation_inverse(2*x.view(x.size(0),-1).T - 1, "sigmoid")
#         x = activation_inverse(x.view(x.size(0),-1).T, "sigmoid")
        y_one_hot = F.one_hot(y, 10).to(device).T
        y_one_hot = 0.94 * y_one_hot + 0.03 * torch.ones(*y_one_hot.shape, device = device)
        _ = model.batch_step(  x, y_one_hot, lr, neural_lr_start, neural_lr_stop, neural_lr_rule,
                               neural_lr_decay_multiplier, neural_dynamic_iterations,
                               optimizer = "adam")

    trn_acc = evaluatePC(  model, train_loader, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                           neural_lr_decay_multiplier,
                           neural_dynamic_iterations, device, printing = False)
    tst_acc = evaluatePC(  model, test_loader, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                           neural_lr_decay_multiplier,
                           neural_dynamic_iterations, device, printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

3000it [01:38, 30.39it/s]
3it [00:00, 27.91it/s]

Epoch : 1, Train Accuracy : 0.9039, Test Accuracy : 0.9025


3000it [01:38, 30.54it/s]
3it [00:00, 28.16it/s]

Epoch : 2, Train Accuracy : 0.9318666666666666, Test Accuracy : 0.93


3000it [01:38, 30.48it/s]
3it [00:00, 28.03it/s]

...

Epoch : 48, Train Accuracy : 0.9791, Test Accuracy : 0.9706


3000it [01:37, 30.68it/s]
3it [00:00, 27.96it/s]

Epoch : 49, Train Accuracy : 0.9792333333333333, Test Accuracy : 0.9707


3000it [01:39, 30.01it/s]


Epoch : 50, Train Accuracy : 0.9793666666666667, Test Accuracy : 0.9705


In [8]:
x, y = next(iter(train_loader))
x, y = x.to(device), y.to(device)
x = activation_inverse(x.view(x.size(0),-1).T, "sigmoid")
y_one_hot = F.one_hot(y, 10).to(device).T
y_one_hot = 0.94 * y_one_hot + 0.03 * torch.ones(*y_one_hot.shape, device = device)

neurons = model.fast_forward(x)
mode = "train"
if mode == "train":
    neurons[-1] = y_one_hot.to(torch.float)
    
neurons = model.run_neural_dynamics( x, y, neurons, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                                            neural_lr_decay_multiplier, neural_dynamic_iterations)

layers = [x] + neurons  # concatenate the input to other layers
layers_after_activation = [list(model.activation_func(layers[jj], model.activation_type)) for jj in range(len(layers))]
error_layers = [(layers[jj+1] - (model.Wff[jj]['weight'] @ layers_after_activation[jj][0] + model.Wff[jj]['bias'])) / model.variances[jj + 1] for jj in range(len(layers) - 1)]


In [9]:
error_layers[2].shape, layers_after_activation[2][0].shape

(torch.Size([10, 20]), torch.Size([500, 20]))

In [10]:
(1/20) * (error_layers[jj] @ layers_after_activation[0][0].T)

NameError: name 'jj' is not defined

In [ ]:
jj = 0
torch.mean(outer_prod_broadcasting(error_layers[jj].T, layers_after_activation[jj][0].T), axis = 0)

In [ ]:
error_layers[0]

In [ ]:
model.Wff[0]["weight"].shape

In [ ]:
model.Wff[0]['weight'].shape

In [ ]:
model.activation_func(x, model.activation_type)[0].shape

In [ ]:
neurons = model.fast_forward(activation_func(x, "sigmoid")[0])
neurons[-1] = y_one_hot.to(torch.float)
neurons = model.run_neural_dynamics(x, y, neurons, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                          neural_lr_decay_multiplier, neural_dynamic_iterations)

In [ ]:
tst_acc = evaluatePC(  model, test_loader, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                                 neural_lr_decay_multiplier,
                                 neural_dynamic_iterations, device, printing = True)

In [ ]:
Wff = model.Wff
Wff[0]

In [ ]:
model.batch_step(x, y_one_hot, lr_start, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                          neural_lr_decay_multiplier, neural_dynamic_iterations)

In [ ]:
model.Wff

In [ ]:
evalua

In [ ]:
neurons = model.fast_forward(activation_func(x, "sigmoid")[0])

In [ ]:
layers_after_activation, error_layers, grads = model.calculate_neural_dynamics_grad(x, y, neurons)

In [ ]:
grads

In [ ]:
len(layers_after_activation), len(error_layers), len(grads)

In [ ]:
layers_after_activation[jj + 1][1].shape

In [ ]:
jj = 0
error_layers[jj + 1].shape, model.Wff[jj + 1]['weight'].shape

In [ ]:
len(layers_after_activation)

In [ ]:
jj = 0
Wff[jj]['weight'] @ layers_after_activation[jj][0]

In [ ]:
variances

In [ ]:
Wff = model.Wff
variances = model.variances
layers = [x] + neurons

[(layers[jj+1] - (Wff[jj]['weight'] @ layers_after_activation[jj][0] + Wff[jj]['bias'])) / variances[jj + 1] for jj in range(len(layers) - 1)]

In [ ]:
def activation_func(x, type_ = "linear"):
    if type_ == "linear":
        f_x = x
        fp_x = torch.ones(*x.shape, device = x.device)
    elif type_ == "tanh":
        f_x = torch.tanh(x)
        fp_x = torch.ones(*x.shape, device = x.device) - f_x ** 2
    elif type_ == "sigmoid":
        ones_vec = torch.ones(*x.shape, device = x.device)
        f_x = 1 / (ones_vec + torch.exp(-x))
        fp_x = f_x * (ones_vec - f_x)
    elif type_ == "relu":
        f_x = torch.maximum(x, torch.tensor([0], device = x.device))
        fp_x = 1 * (x > 0)
    elif type_ == "exp":
        f_x = torch.exp(x)
        fp_x = f_x
    else: # Use linear
        f_x = x
        fp_x = torch.ones(*x.shape, device = x.device)
        
    return f_x, fp_x

In [ ]:
x = torch.randn(3,1, device = "cuda")
x

In [ ]:
x = torch.randn(3,1, device = "cuda")
print(x)
activation_func(x, type_ = "sigmoid")

In [ ]:
device = "cuda"

In [ ]:
(2 * torch.rand(3, 3, requires_grad = False).to(device) - 1) * (4 * np.sqrt(6 / (3 + 3)))